In [1]:
import pandas as pd
import numpy as np
from apyori import apriori


# In[3]:


#loading market basket dataset..

df = pd.read_csv('store_data.csv',header=None)


# In[4]:


df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# In[5]:


## Data Cleaning step

# replacing empty value with 0.
df.fillna(0,inplace=True)


In [3]:





# In[6]:


df.head()


# In[7]:


# Data Pre-processing step

# for using aprori , need to convert data in list format..
# transaction = [['apple','almonds'],['apple'],['banana','apple']]....

transactions = []

for i in range(0,len(df)):
    transactions.append([str(df.values[i,j]) for j in range(0,20) if str(df.values[i,j])!='0'])


# In[8]:


## verifying - by printing the 0th transaction
transactions[0]



['shrimp',
 'almonds',
 'avocado',
 'vegetables mix',
 'green grapes',
 'whole weat flour',
 'yams',
 'cottage cheese',
 'energy drink',
 'tomato juice',
 'low fat yogurt',
 'green tea',
 'honey',
 'salad',
 'mineral water',
 'salmon',
 'antioxydant juice',
 'frozen smoothie',
 'spinach',
 'olive oil']

In [4]:
# In[9]:


## verifying - by printing the 1st transaction
transactions[1]

['burgers', 'meatballs', 'eggs']

In [5]:
# In[10]:


# Call apriori function which requires minimum support, confidance and lift, min length is combination of item default is 2".
rules = apriori(transactions, min_support=0.003, min_confidance=0.2, min_lift=3, min_length=2)

## min_support = 0.003 -> means selecting items with min support of 0.3%
## min_confidance = 0.2 -> means min confidance of 20% 
## min_lift = 3  
## min_length = 2 -> means no. of items in the transaction should be 2


# In[11]:


#it generates a set of rules in a generator file...
rules


<generator object apriori at 0x12826e560>

In [6]:






# In[12]:


# all rules need to be converted in a list..
Results = list(rules)
Results


# In[13]:


# convert result in a dataframe for further operation...
df_results = pd.DataFrame(Results)


# In[14]:


# as we see "order_statistics" , is itself a list so need to be converted in proper format..
df_results.head()


# In[15]:


# keep support in a separate data frame so we can use later.. 
support = df_results.support


# In[16]:


'''
convert orderstatistic in a proper format.
order statistic has lhs => rhs as well rhs => lhs 
we can choose any one for convience.
Let's choose first one which is 'df_results['ordered_statistics'][i][0]'
''' 

#all four empty list which will contain lhs, rhs, confidance and lift respectively.
first_values = []
second_values = []
third_values = []
fourth_value = []

# loop number of rows time and append 1 by 1 value in a separate list.. 
# first and second element was frozenset which need to be converted in list..
for i in range(df_results.shape[0]):
    single_list = df_results['ordered_statistics'][i][0]
    first_values.append(list(single_list[0]))
    second_values.append(list(single_list[1]))
    third_values.append(single_list[2])
    fourth_value.append(single_list[3])


# In[17]:


# convert all four list into dataframe for further operation..
lhs = pd.DataFrame(first_values)
rhs = pd.DataFrame(second_values)

confidance=pd.DataFrame(third_values,columns=['Confidance'])

lift=pd.DataFrame(fourth_value,columns=['lift'])


# In[18]:


# concat all list together in a single dataframe
df_final = pd.concat([lhs,rhs,support,confidance,lift], axis=1)
df_final


# In[19]:


'''
 we have some of place only 1 item in lhs and some place 3 or more so we need to a proper represenation for User to understand. 
 replacing none with ' ' and combining three column's in 1 
 example : coffee,none,none is converted to coffee, ,
'''
df_final.fillna(value=' ', inplace=True)
df_final.head()


# In[20]:


#set column name
df_final.columns = ['lhs',1,'rhs',2,3,'support','confidance','lift']
df_final.head()


# In[21]:


# add all three column to lhs itemset only
df_final['lhs'] = df_final['lhs'] + str(", ") + df_final[1]

df_final['rhs'] = df_final['rhs']+str(", ")+df_final[2] + str(", ") + df_final[3]


# In[22]:


df_final.head()


# In[23]:


#drop columns 1,2 and 3 because now we already appended to lhs column.

df_final.drop(columns=[1,2,3],inplace=True)


# In[24]:




In [7]:
#this is final output. You can sort based on the support lift and confidance..
df_final.head()


# In[25]:


## Showing top 10 items, based on lift.  Sorting in desc order
df_final.sort_values('lift', ascending=False).head(10)

,lhs,rhs,support,confidance,lift
58,"olive oil,","mineral water, whole wheat pasta,",0.003866,0.058704,6.115863
6,"fromage blanc,","honey, ,",0.003333,0.245098,5.164271
49,"ground beef,","spaghetti, tomato sauce,",0.003066,0.031208,4.980600
1,"chicken,","light cream, ,",0.004533,0.075556,4.843951
3,"escalope,","pasta, ,",0.005866,0.073950,4.700812
28,"ground beef,","french fries, herb & pepper,",0.003200,0.032564,4.697422
11,"pasta,","shrimp, ,",0.005066,0.322034,4.506672
23,"ground beef,","herb & pepper, chocolate,",0.003999,0.040706,4.490183
69,"frozen vegetables,","mineral water, chocolate, shrimp",0.003200,0.033566,4.417225
10,"olive oil,","whole wheat pasta, ,",0.007999,0.121457,4.122410
